In [3]:
import src.h5Interface as h5i
import src.reactorInterface as RI
import os
import pickle

from scipy.interpolate import interp1d as interp
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import src.blackboard as blackboard
import src.ka as ka
import src.ka_rp as karp
import src.ka_br as kabr
import src.bb_opt as bb_opt
import src.controller as controller
import time
import src.performance_measure as pm
import math
import h5py

import numpy as np
import pandas as pd

In [4]:
def mean_std(results):
    mean = sum([x for x in results]) / len(results)
    var  = sum(pow(x-mean,2) for x in results) / len(results)
    std  = math.sqrt(var)
    return mean, std

def max_pf_val(results, objective):
    max_ = -1E10
    for core in results.values():
        max_ = core[objective] if core[objective] > max_ else max_
    return max_

def min_pf_val(results, objective):
    min_ = 1E10
    for core in results.values():
        min_ = core[objective] if core[objective] < min_ else min_
    return min_

def obj_range(min_, max_):
    return (max_ - min_)

def objective_range(results, objective):
    return max_pf_val(results, objective) - min_pf_val(results, objective)

def calculate_hvi(pf, ll, ul):
    hvi = pm.hypervolume_indicator(pf, ll, ul)
    return hvi

def calculate_dci(ll, ul, pfs, goal, div):
    dci = pm.diversity_comparison_indicator(ll, ul, pfs, goal=goal, div=div)
    return dci

def read_ga_pf(ga_file, design_variables, objectives, num):
    ga_dir = '/Users/ryanstewart/projects/Dakota_interface/' + ga_file 
    f = open(ga_dir + 'final_{}.dat'.format(num))
    ga_pf = {}

    for line in f:
        split_line = line.split('\t')
        split_line = [float(x) for x in split_line]
        name = 'core_[{}, {}, {}]'.format(split_line[0], split_line[1], split_line[2])
        ga_pf[name] = {}
#        for n, dv in enumerate(design_variables.keys()):
 #           ga_pf[name].update({dv: split_line[n]})
        for n, obj in enumerate(objectives.keys()):
            ga_pf[name].update({obj: split_line[n+len(design_variables)]})

    f.close()
    f = open(ga_dir + 'sfr_moo_bb_comp_2_{}.out'.format(num, 'r'))
    for line in f:
        if 'wall clock' in line:
            split_line = line.split('  ')
            ga_time = float(split_line[-1])
    f.close()
    
    for core in ga_pf.values():
        for obj in core.keys():
            if obj == 'burnup':
                core[obj] = -core[obj]
                
    pf = [[core[obj] for obj in objectives.keys()] for core in ga_pf.values()]
    ga_hvi = 0#calculate_hvi(pf, [x['ll'] for x in objectives.values()], [x['ul'] for x in objectives.values()])
    for core in ga_pf.values():
        for obj in core.keys():
            if obj == 'burnup':
                core[obj] = -core[obj]

    return ga_pf, ga_time, ga_hvi

def read_bb_pf(bb_file, design_variables, objectives, num):
    ns = run_nameserver()
    bb = run_agent(name='bb', base=bb_opt.BbOpt)
    bb.set_attr(archive_name=bb_file+'sfr_bb_{}.h5'.format(num))
    bb.load_h5(panels={2:['new','old'],3:['new','old']})
    lvls = bb.get_attr('abstract_lvls')
    lvl1 = lvls['level 1']    
    lvl3 = {}
    for name in lvls['level 3'].values():
        lvl3.update(name)
    bb_pf = {}
    for name, rx_parameters in lvl1.items():
        bb_pf[name] = {}
#        for num, dv in enumerate(design_variables.keys()):
 #           bb_pf[name].update({dv: lvl3[name]['design variables'][dv]})
        for num, obj in enumerate(objectives.keys()):
            bb_pf[name].update({obj: lvl3[name]['objective functions'][obj]})
    meta_data = lvls['level 100']['final']
    
    ns.shutdown()
    time.sleep(1)
    return bb_pf, meta_data['time'], meta_data['hvi']
    
def compare_results(num_to_compare, ga_dir, bb_dir, objectives, design_variabes, div, goal):
    ga_pfs = {}
    bb_pfs = {}
    full_pf = []
    for i in range(num_to_compare):
        try:
            ga_pf, ga_time, ga_hvi = read_ga_pf(ga_dir, design_variables, objectives, i)        
            full_pf.append(ga_pf)
            ga_pfs[i] = {'pf': ga_pf, 'time': ga_time, 'hvi': ga_hvi}
            print(ga_pfs[i]['hvi'])
        except:
            print('Failed to load ga {}'.format(i))
        bb_pf, bb_time, bb_hvi = read_bb_pf(bb_dir, design_variables, objectives, i)
        full_pf.append(bb_pf)
        bb_pfs[i] = {'pf': bb_pf, 'time': bb_time, 'hvi': bb_hvi}

    
    ll = {obj_name: obj['ll'] for obj_name, obj in objectives.items()}
    ul = {obj_name: obj['ul'] for obj_name, obj in objectives.items()}

#    dci_object = calculate_dci(ll, ul, full_pf, goal, div)
 #   for i in range(num_to_compare):
  #      dci_object.compute_dci(ga_pfs[i]['pf'])
   #     ga_pfs[i].update({'dci': dci_object.dci})
    #    dci_object.compute_dci(bb_pfs[i]['pf'])
     #   bb_pfs[i].update({'dci': dci_object.dci})
    
    results = ['dci', 'hvi', 'time',]
    ga_temp  ={}
    for i in range(num_to_compare):
        ga_temp['time'] = ga_pfs[i]['time']
#        ga_temp['dci'] = ga_pfs[i]['dci']
        ga_temp['hvi'] = ga_pfs[i]['hvi']
    return ga_pfs, bb_pfs

def get_objectives_in_level(lvl):
    obj_dict = {}
    
    for core, values in lvl_3.items():
        if core in lvl:
            dvs = values['design variables']
            core_params = values['objective functions']
            for dv in bb.get_attr('design_variables'):
                if dv in obj_dict.keys():
                    obj_dict[dv].append(dvs[dv])
                else:
                    obj_dict[dv] = [dvs[dv]] 
            for obj in bb.get_attr('objectives').keys():
                if obj in obj_dict.keys():
                    obj_dict[obj].append(core_params[obj])
                else:
                    obj_dict[obj] = [core_params[obj]]
    return obj_dict

def get_non_pareto_sols():
    obj_dict = {}
    
    for core, values in lvl_2.items():
        if core not in lvl_1:
            dvs = lvl_3[core]['design variables']
            core_params = lvl_3[core]['objective functions']
            for dv in bb.get_attr('design_variables'):
                if dv in obj_dict.keys():
                    obj_dict[dv].append(dvs[dv])
                else:
                    obj_dict[dv] = [dvs[dv]] 
            for obj in bb.get_attr('objectives').keys():
                if obj in obj_dict.keys():
                    obj_dict[obj].append(core_params[obj])
                else:
                    obj_dict[obj] = [core_params[obj]]
    return obj_dict

def get_objectives_not_in_level(lvl):
    obj_dict = {}

    for core, values in lvl_3.items():
        if core not in lvl:
            dvs = values['design variables']
            core_params = values['objective functions']
            for dv in bb.get_attr('design_variables'):
                if dv in obj_dict.keys():
                    obj_dict[dv].append(dvs[dv])
                else:
                    obj_dict[dv] = [dvs[dv]]                
            for obj in bb.get_attr('objectives').keys():
                if obj in obj_dict.keys():
                    obj_dict[obj].append(core_params[obj])
                else:
                    obj_dict[obj] = [core_params[obj]]
    return obj_dict

def make_2d_fig(x_obj, y_obj, null=False):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=lvl_1_obj[x_obj], y=lvl_1_obj[y_obj], mode='markers', name='pareto'))
    if null == True:
        fig.add_trace(go.Scatter(x=null_space[x_obj], y=null_space[y_obj], mode='markers', name='null'))
    fig.update_xaxes(title_text=obj_labels[x_obj])
    fig.update_yaxes(title_text=obj_labels[y_obj])
    fig.show()

def make_3d_fig(x_obj, y_obj, z_obj, null=False):
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(x=lvl_1_obj[x_obj], y=lvl_1_obj[y_obj], z=lvl_1_obj[z_obj], mode='markers', name='pareto'))
    if null == True:
        fig.add_trace(go.Scatter3d(x=null_space[x_obj], y=null_space[y_obj], z=null_space[z_obj], mode='markers', name='null'))
    fig.update_layout(scene={'xaxis': {'title': obj_labels[x_obj]}, 'yaxis':{'title': obj_labels[y_obj]},'zaxis':{'title': obj_labels[z_obj]},})
    fig.show()
    
def make_dv_histogram(bins):
    for num, dv in enumerate(bb.get_attr('design_variables').keys()):
        mean, std_dev = mean_std(lvl_1_obj[dv])
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=lvl_1_obj[dv], histnorm='probability',xbins=bins[num]))
        fig.add_shape(dict(
            type="line",
            x0=mean,
            y0=0,
            x1=mean,
            y1=1.0,
            line=dict(
                color="goldenrod",
                width=3)))
    
        layout = go.Layout(annotations = [dict(x=mean,
            y=1,
            xref='x',
            yref='paper',
            text="Mean: {}".format(round(mean,3)),
            showarrow=True,
            arrowhead=7,
            ax=1,
            ay=1)])
        fig.update_layout(layout)
        fig.update_shapes(dict(yref='paper'))
        fig.update_xaxes(title_text=obj_labels[dv])
        fig.update_yaxes(title_text='Probability')
        fig.show()

In [181]:
objective_3d = {'reactivity swing': {'ll':250,  'ul':1000,  'goal':'lt', 'variable type': float},
             'burnup':           {'ll':-80,   'ul':-30,    'goal':'gt', 'variable type': float},
             'pu mass':          {'ll':0,    'ul':1000,  'goal':'lt', 'variable type': float}}
objective_2d = {'reactivity swing': {'ll':250,  'ul':1000,  'goal':'lt', 'variable type': float},
                'burnup':           {'ll':-80,   'ul':-30,    'goal':'gt', 'variable type': float}}
design_variables = {'height':     {'ll': 50.0, 'ul': 100.0, 'variable type': float},
                    'smear':      {'ll': 50.0, 'ul': 80.0, 'variable type': float},
                    'pu_content': {'ll': 0.0,  'ul': 1.0,  'variable type': float}}
div_2d={'reactivity swing':  150, 'burnup':  100}        
goal_2d={'reactivity swing':  'lt', 'burnup':  'gt'} 
div_3d={'reactivity swing':  150, 'burnup':  100, 'pu mass': 100}        
goal_3d={'reactivity swing':  'lt', 'burnup':  'gt', 'pu mass': 'lt'} 
bb_file_2d = 'mc_2d_1E5/'
ga_file_2d = 'GA_BB_2D/'
bb_file_3d = 'mc_3d_1E5/'
#bb_file_3d = 'mc_3d_1E6/'
ga_file_3d = 'GA_BB_3D/'

time_1 = time.time()
ga_, bb_ = compare_results(20, ga_file_3d, bb_file_3d, objective_3d, design_variables, div_3d, goal_3d)
#ga, bb = compare_results(20, ga_file_2d, bb_file_2d, objective_2d, design_variables, div_2d, goal_2d)
print(time.time()-time_1)
a = 1

0
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:16533 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:16533
INFO [2020-10-03 14:42:57.637198] (bb): Loading H5 archive: mc_3d_1E5/sfr_bb_0.h5
NS shut down.
0
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:11982 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:11982
INFO [2020-10-03 14:43:08.436195] (bb): Loading H5 archive: mc_3d_1E5/sfr_bb_1.h5
NS shut down.
0
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:17879 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:17879
INFO [2020-10-03 14:43:26.066361] (bb): Loading H5 archive: mc_3d_1E5/sfr_bb_2.h5
NS shut down.
0
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:15756 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:15756
INFO [2020-10-03 14:43:43.877336] (bb): Loading H5 archive: mc_3d_1E5/sfr_bb_3.h5
NS shut down.
0
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:16723 (127.0.0.1)
URI = PYRO:Pyro.Nam

In [190]:
print(max([x['time'] for x in bb.values()]))
print(max([x['time'] for x in ga.values()]))
a = [x['time'] for x in bb.values()]
print(a)
a.pop(7)
print(mean_std(a))

5710.084428787231
6118.06
[803.7828121185303, 3753.092851161957, 2245.455804824829, 3515.1633820533752, 3582.752044200897, 1316.991132736206, 1712.7940850257874, 5710.084428787231, 505.67740392684937, 2821.892925977707, 1323.7978918552399, 2548.384616136551, 1589.1648461818695, 2486.6557672023773, 1275.2142779827118, 2617.3419938087463, 1699.1853687763214, 1186.0188736915588, 2114.141856908798, 1261.65829205513]
(2018.9034856118653, 923.3035938023277)


# Plotting - 3D

In [22]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_opt.BbOpt)
bb.set_attr(archive_name='mc_2d_1E5/sfr_bb_13.h5')
bb.load_h5(panels={2:['new','old'],3:['new','old']})

objectives_3d = {'reactivity swing': {'ll':250,   'ul':1000,  'goal':'lt', 'variable type': float},
              'burnup':           {'ll':30,   'ul':80,  'goal':'gt', 'variable type': float},
              'pu mass':          {'ll':0,   'ul':1000, 'goal':'lt', 'variable type': float}}
objectives_2d = {'reactivity swing': {'ll':250,   'ul':1000,  'goal':'lt', 'variable type': float},
              'burnup':           {'ll':30,   'ul':80,  'goal':'gt', 'variable type': float}}


bb.set_attr(objectives=objectives_2d)
lvl_3 = {}
lvl_2 = {}

for panel in bb.get_attr('abstract_lvls')['level 3'].values():
    lvl_3.update(panel)
for panel in bb.get_attr('abstract_lvls')['level 2'].values():
    lvl_2.update(panel)

lvl_1 = bb.get_attr('abstract_lvls')['level 1']
print(len(lvl_1))

obj_labels = {'fitness': 'fitness',
              'burnup': 'Burnup (GWD/MTHM)',
              'cycle length': 'Cycle Length(days)',
              'pu mass': 'Pu Mass (kg/year)',
              'reactivity swing' : 'Rx Swing (pcm/month)',
              'height': 'Height (cm)',
              'smear': 'Smear',
              'pu_content': 'Pu Frac.'}
objs = [x for x in bb.get_attr('objectives').keys()]


lvl_1_obj = get_objectives_in_level(lvl_1)
lvl_1_obj['smear'] = [x/100 for x in lvl_1_obj['smear']]
non_pareto = get_non_pareto_sols()
null_space = get_objectives_not_in_level(lvl_2)

#lvl1_df = pd.DataFrame.from_dict(lvl_1_obj)
#null_space_df = pd.DataFrame.from_dict(null_space)
bins = [dict(start=50,end=100,size=5), dict(start=0.50,end=0.80,size=0.015), dict(start=0,end=1.0,size=0.05)]
make_dv_histogram(bins)


#print(lvl1_df)
#fig1 = px.scatter(lvl1_df, x=x_obj, y=y_obj,
#                  labels={x_obj:obj_labels[x_obj], y_obj: obj_labels[y_obj]})
#fig1.show()

    
null = False
#for x_obj in bb.get_attr('objectives'):
#    for y_obj in bb.get_attr('objectives'):
#        if x_obj != y_obj:
#            make_2d_fig(x_obj,y_obj,null=null)

if len(bb.get_attr('objectives')) >= 3:
    x_obj = 'reactivity swing'
    y_obj = 'burnup'
    z_obj = 'pu mass'
#    make_3d_fig(x_obj,y_obj,z_obj,null=null)


x_obj = 'height'
y_obj = 'smear'
z_obj = 'pu_content'
#make_3d_fig(x_obj,y_obj,z_obj,null=null)
ns.shutdown()


Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:16383 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:16383
INFO [2020-10-20 20:42:14.252303] (bb): Loading H5 archive: mc_2d_1E5/sfr_bb_13.h5
133


NS shut down.


# Ploting - 4D Cycle Length

In [3]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_opt.BbOpt)
bb.set_attr(archive_name='sfr_bb_0_4d.h5')
bb.load_h5(panels={2:['new','old'],3:['new','old']})

objectives = {'reactivity swing': {'ll':250, 'ul':1500,  'goal':'lt', 'variable type': float},
              'burnup':           {'ll':20,  'ul':100,    'goal':'gt', 'variable type': float},
              'pu mass':          {'ll':0,    'ul':1000,  'goal':'lt', 'variable type': float},
              'cycle length':     {'ll':60,  'ul':360, 'goal':'gt', 'variable type': float}}
 
bb.set_attr(objectives=objectives)
lvl_3 = {}
lvl_2 = {}

for panel in bb.get_attr('abstract_lvls')['level 3'].values():
    lvl_3.update(panel)
for panel in bb.get_attr('abstract_lvls')['level 2'].values():
    lvl_2.update(panel)

lvl_1 = bb.get_attr('abstract_lvls')['level 1']
print(len(lvl_1))

obj_labels = {'fitness': 'fitness',
              'burnup': 'Burnup (GWD/MTHM)',
              'cycle length': 'Cycle Length (days)',
              'pu mass': 'Pu Mass (kg/year)',
              'reactivity swing' : 'Rx Swing (pcm/month)',
              'height': 'Height (cm)',
              'smear': 'Smear',
              'pu_content': 'Pu Frac.'}
objs = [x for x in bb.get_attr('objectives').keys()]


def get_objectives_in_level(lvl):
    obj_dict = {}
    
    for core, values in lvl_3.items():
        if core in lvl:
            dvs = values['design variables']
            core_params = values['objective functions']
            for dv in bb.get_attr('design_variables'):
                if dv in obj_dict.keys():
                    obj_dict[dv].append(dvs[dv])
                else:
                    obj_dict[dv] = [dvs[dv]] 
            for obj in bb.get_attr('objectives').keys():
                if obj in obj_dict.keys():
                    obj_dict[obj].append(core_params[obj])
                else:
                    obj_dict[obj] = [core_params[obj]]
    return obj_dict

def get_objectives_not_in_level(lvl):
    obj_dict = {}

    for core, values in lvl_3.items():
        if core not in lvl:
            dvs = values['design variables']
            core_params = values['objective functions']
            for dv in bb.get_attr('design_variables'):
                if dv in obj_dict.keys():
                    obj_dict[dv].append(dvs[dv])
                else:
                    obj_dict[dv] = [dvs[dv]]                
            for obj in bb.get_attr('objectives').keys():
                if obj in obj_dict.keys():
                    obj_dict[obj].append(core_params[obj])
                else:
                    obj_dict[obj] = [core_params[obj]]
    return obj_dict


lvl_1_obj = get_objectives_in_level(lvl_2)
null_space = get_objectives_not_in_level(lvl_2)

#lvl1_df = pd.DataFrame.from_dict(lvl_1_obj)
#null_space_df = pd.DataFrame.from_dict(null_space)
#print(lvl1_df)
#fig1 = px.scatter(lvl1_df, x=x_obj, y=y_obj,
#                  labels={x_obj:obj_labels[x_obj], y_obj: obj_labels[y_obj]})
#fig1.show()

def make_2d_fig(x_obj, y_obj):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=lvl_1_obj[x_obj], y=lvl_1_obj[y_obj], mode='markers', name='pareto'))
 #   fig.add_trace(go.Scatter(x=null_space[x_obj], y=null_space[y_obj], mode='markers', name='null'))
    fig.update_xaxes(title_text=obj_labels[x_obj])
    fig.update_yaxes(title_text=obj_labels[y_obj])
    fig.show()

def make_3d_fig(x_obj, y_obj, z_obj, null=False):
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(x=lvl_1_obj[x_obj], y=lvl_1_obj[y_obj], z=lvl_1_obj[z_obj], mode='markers', name='pareto'))
    if null:
        fig.add_trace(go.Scatter3d(x=null_space[x_obj], y=null_space[y_obj], z=null_space[z_obj], mode='markers', name='null'))
    fig.update_layout(scene={'xaxis': {'title': obj_labels[x_obj]}, 'yaxis':{'title': obj_labels[y_obj]},'zaxis':{'title': obj_labels[z_obj]},})
    fig.show()
    
x_obj = 'cycle length'
y_obj = 'burnup'
make_2d_fig(x_obj,y_obj)
x_obj = 'cycle length'
y_obj = 'reactivity swing'
make_2d_fig(x_obj,y_obj)
x_obj = 'burnup'
y_obj = 'reactivity swing'
make_2d_fig(x_obj,y_obj)
x_obj = 'pu mass'
y_obj = 'reactivity swing'
make_2d_fig(x_obj,y_obj)
x_obj = 'pu mass'
y_obj = 'burnup'
make_2d_fig(x_obj,y_obj)
x_obj = 'pu mass'
y_obj = 'cycle length'
make_2d_fig(x_obj,y_obj)
x_obj = 'reactivity swing'
y_obj = 'burnup'
z_obj = 'cycle length'
make_3d_fig(x_obj,y_obj,z_obj)

x_obj = 'height'
y_obj = 'smear'
z_obj = 'pu_content'
make_3d_fig(x_obj,y_obj,z_obj, null=True)

fig1 = px.scatter_3d(x=lvl_1_obj[x_obj], y=lvl_1_obj[y_obj], z=lvl_1_obj[z_obj], color=lvl_1_obj['cycle length'], labels={'x':'height', 'y': 'smear', 'z': 'pu content', 'label': 'fitness'})
fig1.show()
fig1 = px.scatter_3d(x=lvl_1_obj[x_obj], y=lvl_1_obj[y_obj], z=lvl_1_obj[z_obj], color=lvl_1_obj['burnup'], labels={'x':'height', 'y': 'smear', 'z': 'pu content', 'label': 'fitness'})
fig1.show()
fig1 = px.scatter_3d(x=lvl_1_obj[x_obj], y=lvl_1_obj[y_obj], z=lvl_1_obj[z_obj], color=lvl_1_obj['reactivity swing'], labels={'x':'height', 'y': 'smear', 'z': 'pu content', 'label': 'fitness'})
fig1.show()


Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:15073 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:15073


NameError: name 'bb_opt' is not defined

NS shut down.


In [41]:
lvl_3 = {}
lvl_2 = {}
lvl_1 = {}
for num in range(20):
    ns = run_nameserver()
    bb = run_agent(name='bb', base=bb_opt.BbOpt)
    bb.set_attr(archive_name='mc_3d_1E5/sfr_bb_{}.h5'.format(num))
    bb.load_h5(panels={2:['new','old'],3:['new','old']})

    objectives_3d = {'reactivity swing': {'ll':250,   'ul':1000,  'goal':'lt', 'variable type': float},
              'burnup':           {'ll':30,   'ul':80,  'goal':'gt', 'variable type': float},
              'pu mass':          {'ll':0,   'ul':1000, 'goal':'lt', 'variable type': float}}
    objectives_2d = {'reactivity swing': {'ll':250,   'ul':1000,  'goal':'lt', 'variable type': float},
              'burnup':           {'ll':30,   'ul':80,  'goal':'gt', 'variable type': float}}


    bb.set_attr(objectives=objectives_3d)


    for panel in bb.get_attr('abstract_lvls')['level 3'].values():
        lvl_3.update(panel)
    for panel in bb.get_attr('abstract_lvls')['level 2'].values():
        lvl_2.update(panel)

    lvl_1.update(bb.get_attr('abstract_lvls')['level 1'])

    objs = [x for x in bb.get_attr('objectives').keys()]
    if num != 19:
        ns.shutdown()
    time.sleep(1)

lvl_1_obj = get_objectives_in_level(lvl_1)
lvl_1_obj['smear'] = [x/100 for x in lvl_1_obj['smear']]
non_pareto = get_non_pareto_sols()
null_space = get_objectives_not_in_level(lvl_2)

bins = [dict(start=50,end=100,size=5), dict(start=0.50,end=0.80,size=0.015), dict(start=0,end=1.0,size=0.05)]
make_dv_histogram(bins)



Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:12132 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:12132
INFO [2020-10-20 21:04:35.120479] (bb): Loading H5 archive: mc_3d_1E5/sfr_bb_0.h5
NS shut down.
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:14106 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:14106
INFO [2020-10-20 21:04:49.076813] (bb): Loading H5 archive: mc_3d_1E5/sfr_bb_1.h5
NS shut down.
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:19146 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:19146
INFO [2020-10-20 21:05:10.339620] (bb): Loading H5 archive: mc_3d_1E5/sfr_bb_2.h5
NS shut down.
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:18893 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:18893
INFO [2020-10-20 21:05:31.640597] (bb): Loading H5 archive: mc_3d_1E5/sfr_bb_3.h5
NS shut down.
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:18371 (127.0.0.1)
URI = PYRO:Pyro.NameServer@12